In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px


path = 'caminho'
df = pd.read_csv(path)

In [ ]:
df.info()

In [ ]:
#investigar colunas e definir por tipos para analise

categoricals = 

In [ ]:
corr = df.corr(numeric_only = True)

fig = px.imshow(
    corr,
    text_auto = '.4f',
    aspect = 'auto',
    title = 'Correlação entre variáveis continuas (corr de pearson),
    color_continus_scale = 'RdBu'
)

fig

In [ ]:
import pingouin as pg

pvals = corr.corr(method = lambda x, y: pg.corr(x, y)['p-val'].iloc[0])

np.fill_diagonal(pvals.values, np.nan)

fig = px.ishow(
    pvals,
    text_auto='.4f',
    color_continus_scale = 'RedBu'
)

fig

In [ ]:
def check_significance(pvals, alpha=0.05):
    n = len(pvals.columns)
    for i in range(n):
        for j in range(i + 1, n):
            pvalue = pvals.iloc[i, j]
            if pvalue < alpha:
                var1 = pvals.columns[i]
                var2 = pvals.columns[j]
                print(f'Correlação significante entre: {var1} e {var2} (p-valor = {pvalue})')

check_significance(pvals)

In [ ]:
import scipy.stats as stats 
important = variable_types['categorical_encoded']

catImportant = df[important]


def anova(df, cat, num):

    grouped = df.groupby(cat)[num]

    groups = [group.values[~pd.isnull(group.values)] for _, group in grouped]

    f_val, p_val = stats.f_oneway(*groups)
    return p_val

results = []

def testing_anova(df, cat, num):

  for cat_col in categorical_columns:
    if len(df[cat_col].unique()) > 2:
      for num_col in num:
          p_val = anova(df, num_col, cat_col)

          if p_val is not None and p_val < 0.05:
              print(f"Significativo: {num_col} e {cat_col} (p-valor={p_val:.6f})")
              results.append((cat_col, num_col, p_val))

testing_anova(df, categoricals, numerics)

In [ ]:
from scipy.stats import chi2_contingency


cramers_matrix = pd.DataFrame(
    np.nan,
    index=important,
    columns=important
)
important += ['inscrito_actual', 'ev_trad', 'ev_outro', 'apoio', 'asesorias', 'recursado', 'terminado'] 
np.fill_diagonal(cramers_matrix.values, 1.0)


def cramers_v(contingency_table):
    chi2, _, _, _ = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    phi2 = chi2 / n
    r, k = contingency_table.shape
    return np.sqrt(phi2 / min(k-1, r-1))


for i, col1 in enumerate(important):
    for j, col2 in enumerate(important):
        if i < j:
            contingency = pd.crosstab(
                df[col1],
                df[col2]
            )
            cv = cramers_v(contingency.values)
            cramers_matrix.loc[col1, col2] = cv
            cramers_matrix.loc[col2, col1] = cv

fig = px.imshow(
    cramers_matrix,
    text_auto='.4f',
    color_continuous_scale='RdBu_r'
    )

fig.update_layout(
    width=1000,
    height=800,
    margin=dict(l=100, r=100, t=100, b=100),

)

fig